In [1]:
import sys
sys.path.append("../")

In [5]:
from data_tools.containers import StocksData, StocksTarget
from data_tools.samplers import SimpleModelPreprocessor
from targets.target_function import ClosePriceTargetFunction
from features import IndicatorsFeaturesList

Firstly, we need to read the data and add features to it. `StocksData` class is responsible for reading the stock data from file. `FeaturesList` class contains list of features which can be added to our AAPL data.

In [3]:
stocks_data = StocksData(files=["../data/market_data/AAPL.csv"])
stocks_data = IndicatorsFeaturesList().preprocess(stocks_data)

Let's check internal DataFrame.

In [4]:
stocks_data.get_data().head()

,high,low,open,close,volume,ticker,close_5_mean,close_10_mean,close_20_mean,close_5_var,close_10_var,close_20_var
date,,,,,,,,,,,,
2021-01-25 14:30:00,145.080002,142.940002,143.125000,144.229996,46354729.0,AAPL,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-25 15:30:00,144.320007,136.539993,144.229996,141.220001,37087128.0,AAPL,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-25 16:30:00,142.949997,140.910004,141.220001,141.190002,18948572.0,AAPL,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-25 17:30:00,143.289993,141.039993,141.179993,142.989899,14505566.0,AAPL,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-25 18:30:00,143.250000,141.789993,143.009995,142.582001,11219998.0,AAPL,142.44238,NaN,NaN,1.644375,NaN,NaN


Now, we need to define our targets. `Target` is a class which contains target data `StocksData`. It is created when we call `get` method of `TargetFunction` class. Also, you can create `Target` from its static method `from_target_function`.

In [6]:
stocks_target = StocksTarget.from_target_function(stocks_data, ClosePriceTargetFunction(bars_count=3))

In [7]:
stocks_target.get_data().head()

,close_target_3_bars,ticker
date,,
2021-01-25 14:30:00,142.989899,AAPL
2021-01-25 15:30:00,142.582001,AAPL
2021-01-25 16:30:00,142.070007,AAPL
2021-01-25 17:30:00,142.899994,AAPL
2021-01-25 18:30:00,142.212097,AAPL


At this moment we have to do final preprocessing step in order to prepare our data for training a model. We can do it with one of `StockDataModelPreprocessor` class inheritors. 

In [8]:
train_data, test_data, train_target, test_target = SimpleModelPreprocessor.split(stocks_data, stocks_target)

TypeError: 'Timestamp' object is not callable

Finally, we can train our model and measure MAPE on the test sample.

In [6]:
model = GBMRegressionModel()
model.fit(X_train, y_train, verbose=False)

predictions = model.predict(X_test)

metric = mean_absolute_percentage_error(y_test, predictions)
print(f"MAPE: {metric}")

MAPE: 0.10311060786645385
